## Lab 1.3 - Predicting Real Estate Data in St. Petersburg
We have data from Yandex.Realty classified https://realty.yandex.ru containing real estate listings for apartments in St. Petersburg and Leningrad Oblast from 2016 till the middle of August 2018. In this Lab you'll learn how to apply machine learning algorithms to solve business problems. Accurate price prediction can help to find fraudsters automatically and help Yandex.Realty users to make better decisions when buying and selling real estate.

Using python with machine learning algotithms is the #1 option for prototyping solutions among data scientists today. We'll take a look at it in this lab.

### Main objectives
After successful completion of the lab work students will be able to:
-	Apply machine learning for solving price prediction problem
-   Calculate metrics which can help us find out whether our machine learning model is ready for production

### Tasks
-	Encode dataset
-	Split dataset to train and validation datasets
-	Apply decision tree algorithm to build ML (machine learning) model for price predictions
-   Calculate metrics
-   Try other algorithms and factors to get a better solution 


### 1. Load data with real estate prices

In [2]:
!python -m pip install scikit-learn --upgrade!pip install --upgrade pip
!pip install sklearn_pandas


Usage:   
  /opt/conda/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /opt/conda/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /opt/conda/bin/python -m pip install [options] [-e] <vcs project url> ...
  /opt/conda/bin/python -m pip install [options] [-e] <local project path> ...
  /opt/conda/bin/python -m pip install [options] <archive url/path> ...

no such option: --upgrade!pip
  Using cached sklearn_pandas-2.2.0-py2.py3-none-any.whl (10 kB)

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.style as style
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)
style.use('fivethirtyeight')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn_pandas import DataFrameMapper
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

In [6]:
rent_df_cleaned = pd.read_csv('cleaned_dataset.csv')

In [7]:
rent_df_cleaned.head()

,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,agent_fee,renovation,last_price_log
0,2015-01-24T00:00:00+03:00,2016-01-19T00:00:00+03:00,20000.0,7,0,1,0,28.0,100.0,3.0,9.903488
1,2015-11-17T00:00:00+03:00,2016-03-04T00:00:00+03:00,24000.0,4,0,2,0,59.0,100.0,3.0,10.085809
2,2015-11-17T00:00:00+03:00,2016-04-24T00:00:00+03:00,18000.0,6,0,1,0,36.0,100.0,3.0,9.798127
3,2016-02-04T00:00:00+03:00,2016-02-28T00:00:00+03:00,18000.0,8,0,1,0,39.0,90.0,0.0,9.798127
4,2016-02-28T00:00:00+03:00,2016-04-02T00:00:00+03:00,19000.0,3,0,1,0,36.0,50.0,11.0,9.852194


In [8]:
rent_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155391 entries, 0 to 155390
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   first_day_exposition  155391 non-null  object 
 1   last_day_exposition   155391 non-null  object 
 2   last_price            155391 non-null  float64
 3   floor                 155391 non-null  int64  
 4   open_plan             155391 non-null  int64  
 5   rooms                 155391 non-null  int64  
 6   studio                155391 non-null  int64  
 7   area                  155391 non-null  float64
 8   agent_fee             122840 non-null  float64
 9   renovation            155391 non-null  float64
 10  last_price_log        155391 non-null  float64
dtypes: float64(5), int64(4), object(2)
memory usage: 13.0+ MB


### Self-control stops
1. Compete with other teams to create the best solution. You can play with factors and algorithm parameters to come up with it.

In [9]:
rent_df_cleaned['f_day_exposition'] = pd.to_datetime(rent_df_cleaned.first_day_exposition)

In [10]:
rent_df_cleaned['l_day_exposition'] = pd.to_datetime(rent_df_cleaned.last_day_exposition)

In [11]:
rent_df_cleaned['offer_time'] = (rent_df_cleaned['l_day_exposition'] - rent_df_cleaned['f_day_exposition']).dt.days

In [12]:
rent_df_cleaned.drop(columns=['last_price_log'], inplace = True)
rent_df_cleaned.drop(columns=['f_day_exposition'], inplace = True)
rent_df_cleaned.drop(columns=['l_day_exposition'], inplace = True)

In [13]:
rent_df_cleaned.head()

,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,agent_fee,renovation,offer_time
0,2015-01-24T00:00:00+03:00,2016-01-19T00:00:00+03:00,20000.0,7,0,1,0,28.0,100.0,3.0,360
1,2015-11-17T00:00:00+03:00,2016-03-04T00:00:00+03:00,24000.0,4,0,2,0,59.0,100.0,3.0,108
2,2015-11-17T00:00:00+03:00,2016-04-24T00:00:00+03:00,18000.0,6,0,1,0,36.0,100.0,3.0,159
3,2016-02-04T00:00:00+03:00,2016-02-28T00:00:00+03:00,18000.0,8,0,1,0,39.0,90.0,0.0,24
4,2016-02-28T00:00:00+03:00,2016-04-02T00:00:00+03:00,19000.0,3,0,1,0,36.0,50.0,11.0,34


In [14]:
rent_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155391 entries, 0 to 155390
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   first_day_exposition  155391 non-null  object 
 1   last_day_exposition   155391 non-null  object 
 2   last_price            155391 non-null  float64
 3   floor                 155391 non-null  int64  
 4   open_plan             155391 non-null  int64  
 5   rooms                 155391 non-null  int64  
 6   studio                155391 non-null  int64  
 7   area                  155391 non-null  float64
 8   agent_fee             122840 non-null  float64
 9   renovation            155391 non-null  float64
 10  offer_time            155391 non-null  int64  
dtypes: float64(4), int64(5), object(2)
memory usage: 13.0+ MB


In [12]:
#split dataset to train and test samples
train_df = rent_df_cleaned[(rent_df_cleaned.first_day_exposition >= '2018-01-01') 
                          & (rent_df_cleaned.first_day_exposition < '2018-04-01')]

In [13]:
len(train_df)

16974

In [14]:
test_df = rent_df_cleaned[(rent_df_cleaned.first_day_exposition >= '2018-04-01') 
                          & (rent_df_cleaned.first_day_exposition < '2018-06-01')]

In [15]:
len(test_df)

14974

In [16]:
holdout_df = rent_df_cleaned[rent_df_cleaned.first_day_exposition >= '2018-06-01']

In [17]:
len(holdout_df)

21112

In [18]:
train_df.head()

,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,agent_fee,renovation,offer_time
8,2018-01-05T00:00:00+03:00,2018-01-16T00:00:00+03:00,26000.0,12,0,1,0,36.0,NaN,0.0,11
24,2018-01-20T00:00:00+03:00,2018-02-28T00:00:00+03:00,17500.0,9,0,1,0,32.0,50.0,1.0,39
25,2018-02-09T00:00:00+03:00,2018-03-03T00:00:00+03:00,16000.0,4,0,1,0,38.0,90.0,0.0,22
26,2018-03-19T00:00:00+03:00,2018-04-18T00:00:00+03:00,22000.0,12,0,1,0,32.0,NaN,0.0,30
27,2018-03-26T00:00:00+03:00,2018-03-30T00:00:00+03:00,20000.0,5,0,1,0,32.0,50.0,0.0,4


In [19]:
train_df.drop(columns=['first_day_exposition','last_day_exposition'], inplace=True)
test_df.drop(columns=['first_day_exposition','last_day_exposition'], inplace=True)

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [20]:
train_df.head()

,last_price,floor,open_plan,rooms,studio,area,agent_fee,renovation,offer_time
8,26000.0,12,0,1,0,36.0,NaN,0.0,11
24,17500.0,9,0,1,0,32.0,50.0,1.0,39
25,16000.0,4,0,1,0,38.0,90.0,0.0,22
26,22000.0,12,0,1,0,32.0,NaN,0.0,30
27,20000.0,5,0,1,0,32.0,50.0,0.0,4


In [21]:
X_train = train_df.drop('last_price', axis=1)
y_train = train_df['last_price']
X_valid = test_df.drop('last_price', axis=1)
y_valid = test_df['last_price']
y_train= y_train.values.reshape(-1,1)
y_valid= y_valid.values.reshape(-1,1)
print(X_train)

        floor  open_plan  rooms  studio  area  agent_fee  renovation  offer_time
8          12          0      1       0  36.0        NaN         0.0          11
24          9          0      1       0  32.0       50.0         1.0          39
25          4          0      1       0  38.0       90.0         0.0          22
26         12          0      1       0  32.0        NaN         0.0          30
27          5          0      1       0  32.0       50.0         0.0           4
...       ...        ...    ...     ...   ...        ...         ...         ...
154884      2          0      1       0  46.0        0.0        11.0         143
154921      6          0      2       0  50.0       90.0        11.0         176
155028      4          0      3       0  66.0        0.0        11.0         165
155079      4          0      2       0  52.0      100.0         0.0         132
155271      1          0      2       0  46.0       50.0         0.0         193

[16974 rows x 8 columns]


In [22]:
numeric_features = ['area', 'offer_time', 'agent_fee' ] 
nominal_features = ['renovation','open_plan', 'rooms', 'floor', 'studio']

In [23]:
mapper = DataFrameMapper([([feature], SimpleImputer()) for feature in numeric_features] +\
                         [([feature], OneHotEncoder(handle_unknown='ignore')) for feature in nominal_features],
                             df_out=True)

pipeline = Pipeline(steps = [('preprocessing', mapper), 
                             ('scaler', StandardScaler()),
                             ('LinearRegression', LinearRegression())])
                             
pipeline

Pipeline(steps=[('preprocessing',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['area'], SimpleImputer()),
                                           (['offer_time'], SimpleImputer()),
                                           (['agent_fee'], SimpleImputer()),
                                           (['renovation'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['open_plan'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['rooms'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['floor'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['studio'],
                                            OneHotEnco

In [24]:
result = mapper.fit_transform(rent_df_cleaned)
result.head()

,area,offer_time,agent_fee,renovation_x0_0.0,renovation_x0_1.0,renovation_x0_2.0,renovation_x0_3.0,renovation_x0_4.0,renovation_x0_5.0,renovation_x0_6.0,renovation_x0_7.0,renovation_x0_8.0,renovation_x0_10.0,renovation_x0_11.0,open_plan_x0_0,open_plan_x0_1,rooms_x0_0,rooms_x0_1,rooms_x0_2,rooms_x0_3,rooms_x0_4,rooms_x0_5,floor_x0_1,floor_x0_2,floor_x0_3,floor_x0_4,floor_x0_5,floor_x0_6,floor_x0_7,floor_x0_8,floor_x0_9,floor_x0_10,floor_x0_11,floor_x0_12,floor_x0_13,floor_x0_14,floor_x0_15,floor_x0_16,floor_x0_17,floor_x0_18,floor_x0_19,floor_x0_20,floor_x0_21,floor_x0_22,floor_x0_23,floor_x0_24,floor_x0_25,floor_x0_26,floor_x0_27,floor_x0_28,floor_x0_29,floor_x0_30,floor_x0_32,floor_x0_33,floor_x0_34,floor_x0_35,floor_x0_36,studio_x0_0,studio_x0_1
0,28.0,360.0,100.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,59.0,108.0,100.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,36.0,159.0,100.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,39.0,24.0,90.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,36.0,34.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [25]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['area'], SimpleImputer()),
                                           (['offer_time'], SimpleImputer()),
                                           (['agent_fee'], SimpleImputer()),
                                           (['renovation'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['open_plan'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['rooms'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['floor'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['studio'],
                                            OneHotEnco

In [26]:
train_pred = pipeline.predict(X_train)
rmse = mean_squared_error(y_true=y_train, y_pred = train_pred, squared=False)
mape = mean_absolute_percentage_error(y_true=y_train, y_pred=train_pred)
accuracy = pipeline.score(X_train,y_train)


print (f'LR RMSE train = {round(rmse, 3)}')
print (f'LR MAPE train = {round(mape, 3)}')
print('Accuracy = ', accuracy*100,'%')

LR RMSE train = 10309.222
LR MAPE train = 0.22
Accuracy =  59.90070722938139 %


In [27]:
test_pred = pipeline.predict(X_valid)
rmse = mean_squared_error(y_true=y_valid, y_pred = test_pred, squared=False)
mape = mean_absolute_percentage_error(y_true=y_valid, y_pred=test_pred)
accuracy = pipeline.score(X_valid,y_valid)

print (f'LR RMSE test = {round(rmse, 3)}')
print (f'LR MAPE test = {round(mape, 3)}')
print('Accuracy = ', accuracy*100,'%')

LR RMSE test = 11318.566
LR MAPE test = 0.23
Accuracy =  61.11019540630518 %


In [28]:
mapper = DataFrameMapper([([feature], SimpleImputer()) for feature in numeric_features] +\
                         [([feature], OneHotEncoder(handle_unknown='ignore')) for feature in nominal_features],
                             df_out=True)

pipeline = Pipeline(steps = [('preprocessing', mapper), 
                             ('scaler', StandardScaler()),
                             ('decision_tree', DecisionTreeRegressor(max_depth=10, min_samples_leaf=8, max_features=4))])
                             
pipeline

Pipeline(steps=[('preprocessing',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['area'], SimpleImputer()),
                                           (['offer_time'], SimpleImputer()),
                                           (['agent_fee'], SimpleImputer()),
                                           (['renovation'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['open_plan'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['rooms'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['floor'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['studio'],
                                            OneHotEnco

In [29]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['area'], SimpleImputer()),
                                           (['offer_time'], SimpleImputer()),
                                           (['agent_fee'], SimpleImputer()),
                                           (['renovation'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['open_plan'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['rooms'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['floor'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['studio'],
                                            OneHotEnco

In [30]:
train_pred = pipeline.predict(X_train)
rmse = mean_squared_error(y_true=y_train, y_pred = train_pred, squared=False)
mape = mean_absolute_percentage_error(y_true=y_train, y_pred=train_pred)
accuracy = pipeline.score(X_train,y_train)


print (f'RMSE train = {round(rmse, 3)}')
print (f'MAPE train = {round(mape, 3)}')
print('Accuracy = ', accuracy*100,'%')

RMSE train = 10654.852
MAPE train = 0.208
Accuracy =  57.16687748796476 %


In [31]:
test_pred = pipeline.predict(X_valid)
rmse = mean_squared_error(y_true=y_valid, y_pred = test_pred, squared=False)
mape = mean_absolute_percentage_error(y_true=y_valid, y_pred=test_pred)
accuracy = pipeline.score(X_valid,y_valid)

print (f'RMSE test = {round(rmse, 3)}')
print (f'MAPE test = {round(mape, 3)}')
print('Accuracy = ', accuracy*100,'%')

RMSE test = 12055.573
MAPE test = 0.218
Accuracy =  55.880697614429565 %


In [32]:
mapper = DataFrameMapper([([feature], SimpleImputer()) for feature in numeric_features] +\
                         [([feature], OneHotEncoder(handle_unknown='ignore')) for feature in nominal_features],
                             df_out=True)

pipeline = Pipeline(steps = [('preprocessing', mapper), 
                             ('scaler', StandardScaler()),
                             ('random_forest', RandomForestRegressor(random_state=99))])
pipeline

Pipeline(steps=[('preprocessing',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['area'], SimpleImputer()),
                                           (['offer_time'], SimpleImputer()),
                                           (['agent_fee'], SimpleImputer()),
                                           (['renovation'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['open_plan'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['rooms'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['floor'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['studio'],
                                            OneHotEnco

In [33]:
pipeline.fit(X_train, y_train)

/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('preprocessing',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['area'], SimpleImputer()),
                                           (['offer_time'], SimpleImputer()),
                                           (['agent_fee'], SimpleImputer()),
                                           (['renovation'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['open_plan'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['rooms'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['floor'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['studio'],
                                            OneHotEnco

In [34]:
train_pred = pipeline.predict(X_train)
rmse = mean_squared_error(y_true=y_train, y_pred = train_pred, squared=False)
mape = mean_absolute_percentage_error(y_true=y_train, y_pred=train_pred)
accuracy = pipeline.score(X_train,y_train)


print (f'RMSE train = {round(rmse, 3)}')
print (f'MAPE train = {round(mape, 3)}')
print('Accuracy = ', accuracy*100,'%')

RMSE train = 4056.847
MAPE train = 0.08
Accuracy =  93.7904206471246 %


In [35]:
test_pred = pipeline.predict(X_valid)
rmse = mean_squared_error(y_true=y_valid, y_pred = test_pred, squared=False)
mape = mean_absolute_percentage_error(y_true=y_valid, y_pred=test_pred)
accuracy = pipeline.score(X_valid,y_valid)

print (f'RMSE test = {round(rmse, 3)}')
print (f'MAPE test = {round(mape, 3)}')
print('Accuracy = ', accuracy*100,'%')

RMSE test = 11431.507
MAPE test = 0.22
Accuracy =  60.33021500917766 %


In [36]:
from sklearn_pandas import DataFrameMapper
from sklearn.impute import SimpleImputer
mapper = DataFrameMapper([([feature], SimpleImputer()) for feature in numeric_features] +\
                         [([feature], OneHotEncoder(handle_unknown='ignore')) for feature in nominal_features],
                             df_out=True)

pipeline = Pipeline(steps = [('preprocessing', mapper), 
                             ('scaler', StandardScaler()),
                             ('xgb', xgb.XGBRegressor(objective="reg:linear", random_state=99))])
pipeline

Pipeline(steps=[('preprocessing',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['area'], SimpleImputer()),
                                           (['offer_time'], SimpleImputer()),
                                           (['agent_fee'], SimpleImputer()),
                                           (['renovation'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['open_plan'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['rooms'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['floor'],
                                            O...
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=No

In [37]:
pipeline.fit(X_train, y_train)

[15:55:11] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(steps=[('preprocessing',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['area'], SimpleImputer()),
                                           (['offer_time'], SimpleImputer()),
                                           (['agent_fee'], SimpleImputer()),
                                           (['renovation'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['open_plan'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['rooms'],
                                            OneHotEncoder(handle_unknown='ignore')),
                                           (['floor'],
                                            O...
                              feature_types=None, gamma=0, gpu_id=-1,
                              grow_policy='depthwise', importance_type=None,

In [38]:
train_pred = pipeline.predict(X_train)
rmse = mean_squared_error(y_true=y_train, y_pred = train_pred, squared=False)
mape = mean_absolute_percentage_error(y_true=y_train, y_pred=train_pred)
accuracy = pipeline.score(X_train,y_train)


print (f'RMSE train = {round(rmse, 3)}')
print (f'MAPE train = {round(mape, 3)}')
print('Accuracy = ', accuracy*100,'%')

RMSE train = 6548.027
MAPE train = 0.162
Accuracy =  83.8227061344956 %


In [39]:
test_pred = pipeline.predict(X_valid)
rmse = mean_squared_error(y_true=y_valid, y_pred = test_pred, squared=False)
mape = mean_absolute_percentage_error(y_true=y_valid, y_pred=test_pred)
accuracy = pipeline.score(X_valid,y_valid)

print (f'RMSE test = {round(rmse, 3)}')
print (f'MAPE test = {round(mape, 3)}')
print('Accuracy = ', accuracy*100,'%')

RMSE test = 11262.311
MAPE test = 0.204
Accuracy =  61.49581308774685 %


In [40]:
test_pred

array([21045.004, 25907.129, 51685.703, ..., 37575.45 , 20018.14 ,
       20862.496], dtype=float32)

In [42]:
import joblib

In [43]:
model_file = 'model.pkl'

joblib.dump(pipeline, model_file)

['model.pkl']